*Copyright (c) Microsoft Corporation. All rights reserved.*


*Licensed under the MIT License.*

# Text Classification of Yahoo Answers using BERT


In [1]:
import sys
sys.path.append("../../")
import os
import pandas as pd
import utils_nlp.dataset.yahoo_answers as ya_dataset
from utils_nlp.eval.classification import eval_classification
from utils_nlp.bert.sequence_classification import SequenceClassifier
from utils_nlp.bert.common import Language, Tokenizer
import torch
import torch.nn as nn
import numpy as np

In [2]:
DATA_FOLDER = "../../../.../temp"
TRAIN_FILE = "yahoo_answers_csv/train.csv"
TEST_FILE = "yahoo_answers_csv/test.csv"
BERT_CACHE_DIR = "../../../temp"
MAX_LEN = 100
BATCH_SIZE = 32
DEVICE = "gpu"
UPDATE_EMBEDDINGS = False
NUM_EPOCHS = 1
NUM_ROWS_TRAIN = 15000  # number of training examples to read
NUM_ROWS_TEST = 10000  # number of test examples to read

## Download Dataset

In [4]:
ya_dataset.download(DATA_FOLDER)

## Read Dataset

In [ ]:
# read data
df_train = ya_dataset.read_data(os.path.join(DATA_FOLDER, TRAIN_FILE), nrows=NUM_ROWS_TRAIN)
df_test = ya_dataset.read_data(os.path.join(DATA_FOLDER, TEST_FILE), nrows=NUM_ROWS_TEST)

# get labels
labels_train = ya_dataset.get_labels(df_train)
labels_test = ya_dataset.get_labels(df_test)

num_labels = len(np.unique(labels_train))

# get text
text_train = ya_dataset.get_text(df_train)
text_test = ya_dataset.get_text(df_test)

## Tokenize and Preprocess

In [ ]:
tokenizer = Tokenizer(Language.ENGLISH, to_lower=False, cache_dir=BERT_CACHE_DIR)

# tokenize
tokens_train = tokenizer.tokenize(text_train)
tokens_test = tokenizer.tokenize(text_test)

# get BERT-format tokens (padded and truncated)
tokens_train, mask_train = tokenizer.preprocess_classification_tokens(tokens_train, MAX_LEN)
tokens_test, mask_test = tokenizer.preprocess_classification_tokens(tokens_test, MAX_LEN)

## Create Model


In [ ]:
classifier = SequenceClassifier(
    language=Language.ENGLISH, num_labels=num_labels, cache_dir=BERT_CACHE_DIR
)

In [ ]:
# train
classifier.fit(
    tokens=tokens_train,
    input_mask=mask_train,
    labels=labels_train,    
    device=DEVICE,
    use_multiple_gpus=True,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=True,
)

## Score Test Set

In [ ]:
preds = classifier.predict(tokens=tokens_test, input_mask=mask_test, device=DEVICE, batch_size=BATCH_SIZE)

## Evaluate Results

In [ ]:
# eval
eval_results = eval_classification(labels_test, preds)
print("\n accuracy: {}".format(eval_results["accuracy"]))
pd.DataFrame(eval_results)[["precision", "recall", "f1"]]